In [1]:
import boto3
import sagemaker

In [2]:
role = 'arn:aws:iam::638608113287:role/service-role/AmazonSageMaker-ExecutionRole-20180731T132167'
sess = sagemaker.Session()

In [3]:
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name

In [4]:
image = '638608113287.dkr.ecr.us-east-1.amazonaws.com/faster-rcnn:1.0'
train_instance_type = 'ml.p3.16xlarge'
instance_count = 1
output_path="s3://model-artifacts-alkymi/faster-rcnn/"
data_location = "s3://training-data-alkymi/pageseg/2019-06-22_04:18:18.376207"
# data_location = "s3://training-data-alkymi/pageseg/7"
hyperparameters = {
  "batch_size": "32",
  "epochs":"100",
  "lr": "0.0001",
  "lr_decay_gamma": "0.1",
  "lr_patience": "2",
  "patience": "4",
  "imdb_name": "pdfpages",
  "num_workers": "8",
  "USE_FLIPPED": "False"
}

metric_definitions = [
    {'Name': 'train loss', 'Regex': 'train loss: (.*?);'},
    {'Name': 'train rpn_cls', 'Regex': 'train rpn_cls: (.*?);'},
    {'Name': 'train rpn_box', 'Regex': 'train rpn_box: (.*?);'},
    {'Name': 'train rcnn_cls', 'Regex': 'train rcnn_cls: (.*?);'},
    {'Name': 'train rcnn_box', 'Regex': 'train rcnn_box: (.*?);'},
    {'Name': 'val loss', 'Regex': 'val loss: (.*?);'},
    {'Name': 'val rpn_cls', 'Regex': 'val rpn_cls: (.*?);'},
    {'Name': 'val rpn_box', 'Regex': 'val rpn_box: (.*?);'},
    {'Name': 'val rcnn_cls', 'Regex': 'val rcnn_cls: (.*?);'},
    {'Name': 'val rcnn_box', 'Regex': 'val rcnn_box: (.*?);'}]

In [5]:
estimator = sagemaker.estimator.Estimator(image,
                                          role, instance_count, train_instance_type,
                                          output_path=output_path,
                                          sagemaker_session=sess, 
                                          hyperparameters=hyperparameters,
                                          metric_definitions=metric_definitions,m)

In [12]:
estimator.fit(data_location)

2019-06-27 13:07:41 Starting - Starting the training job...
2019-06-27 13:07:44 Starting - Launching requested ML instances............
2019-06-27 13:09:47 Starting - Preparing the instances for training......
2019-06-27 13:10:54 Downloading - Downloading input data..........................................................................................................................................
2019-06-27 13:34:26 Training - Downloading the training image.........
2019-06-27 13:35:59 Training - Training image download completed. Training in progress.
Starting the training.
hyperparameters: {'imdb_name': 'pdfpages', 'batch_size': '32', 'lr_patience': '2', 'USE_FLIPPED': 'False', 'lr': '0.0001', 'patience': '4', 'num_workers': '8', 'epochs': '100', 'lr_decay_gamma': '0.1'}
Loaded dataset `pdfpages_train` for training
Set proposal method: gt
Appending horizontally-flipped training examples...
done
Preparing training data...
done
before filtering, there are 96330 images...
after fil

EndpointConnectionError: Could not connect to the endpoint URL: "https://api.sagemaker.us-east-1.amazonaws.com/"

In [7]:
training_job_name = 'faster-rcnn-2019-06-27-13-07-41-470'
sagemaker.estimator.Estimator.attach(training_job_name, sagemaker_session=sess)

2019-06-27 13:35:59 Starting - Preparing the instances for training
2019-06-27 13:35:59 Downloading - Downloading input data
2019-06-27 13:35:59 Training - Training image download completed. Training in progress.Starting the training.
hyperparameters: {'imdb_name': 'pdfpages', 'batch_size': '32', 'lr_patience': '2', 'USE_FLIPPED': 'False', 'lr': '0.0001', 'patience': '4', 'num_workers': '8', 'epochs': '100', 'lr_decay_gamma': '0.1'}
Loaded dataset `pdfpages_train` for training
Set proposal method: gt
Appending horizontally-flipped training examples...
done
Preparing training data...
done
before filtering, there are 96330 images...
after filtering, there are 96330 images...
Loaded dataset `pdfpages_validation` for training
Set proposal method: gt
Appending horizontally-flipped training examples...
done
Preparing training data...
done
Loading pretrained weights from data/pretrained_model/resnet101_caffe.pth
/opt/conda/lib/python3.6/site-packages/torch/nn/parallel/_functions.py:61: UserWa

ClientError: An error occurred (InvalidSignatureException) when calling the GetLogEvents operation: Signature expired: 20190627T190025Z is now earlier than 20190627T194901Z (20190627T195401Z - 5 min.)

In [ ]:
deploy_instance_type = 'ml.p2.xlarge'
predictor = estimator.deploy(initial_instance_count=1, instance_type=deploy_instance_type)

In [ ]:
import requests
import json
from io import BytesIO
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
file_path = '../data/wf_celg_report.pdf'

with open(file_path, 'rb') as f:
    doc_data = f.read()
params = {'page': 0}
page_image_response = requests.get('https://pdf-service.alkymi.cloud/getPageImage', 
                                   params=params, data=doc_data)
prediction_response = predictor.predict(page_image_response.content)
pred = json.loads(prediction_response)['pred']
img_bytes = BytesIO(page_image_response.content)
img = Image.open(img_bytes)

fig, ax = plt.subplots(figsize=(8.5, 11))
plt.axis('off')

box_type_to_color = {'text':'r', 'graphical_chart':'g', 'structured_data':'b'}
for box_type, boxes in pred.items():
    color = box_type_to_color[box_type]
    for box in boxes:
        rect = patches.Rectangle((float(box[0]), float(box[1])),
                                 float(box[2]) - float(box[0]),
                                 float(box[3]) - float(box[1]),
                                 linewidth=1,
                                 edgecolor=color,
                                 facecolor='none')
        ax.add_patch(rect)
        ax.annotate(round(box[4], 3), 
                    (float(box[0]), float(box[1])), 
                    color=color, 
                    fontsize=12, ha='center', va='center')

ax.imshow(img)
